# <center> Workshop 2: Implement DPos (Delegated Proof of Stake) based blockchain </center>


###### <b> Translated by: </b> BENADDOU Ilyas - BENDOUDA Abir - ELKAOUNI Yousra - ELKARMY Othmane
###### <b> Source code:</b> <b> https://github.com/csunny/dpos </b>

### Summary
- Theory introduction
    - Application
- Implementation
    - Architecture design
    - Code
        - BlockChain module implementation
        - P2P module implementation

# Theory introduction
DPoS (Delegated Proof of Stake) is a new consensus algorithm applied in the field of blockchain technology. In the DPoS algorithm, blocks are produced by a fixed number of producers, and these producers are called producers or witnesses. The BP (Block Producer) or super node that is often heard now is actually the block producer in the DPoS algorithm. Users in the network can vote for these producers to replace themselves with the right to perform block production. The mechanism is similar to the parliamentary system or the people's congress system.

### Application
Since the DPoS algorithm was proposed by BM, it has been applied to blockchain projects such as BitShares (bit shares), Steemit, Lisk, Ark, EOS, and NAS. It can be said that it is one of the few distributed consensus algorithms that are currently proven to be effective.



# Implementation
Before starting the code implementation, first sort out the code structure and design logic. It should be emphasized that the DPoS algorithm implemented here is only a simple implementation.

### Architecture design
- BlockChain module
    - Block Production
    - Block verification
- P2P module
    - Create a P2P node
    - Handling Node Connections
    - Synchronize block information
- Tools module
    - Vote
- Dpos module
    - Build a super node
    - Select nodes (select nodes from super nodes to produce Block)

### Code
According to the above architecture design, let's implement the above modules in turn and test them.

##### <b> BlockChain module implementation </b>

In [ ]:
// implement a simple p2p blockchain which use dpos algorithm.
// this file just for a simple block generate and validate.
// This file is created by magic at 2018-9-2

package dpos

import (
    "crypto/sha256"
    "encoding/hex"
    "time"
)

var BlockChain []Block

type Block struct {
    Index     int
    Timestamp string
    BPM       int
    Hash      string
    PrevHash  string
    validator string
}

func CaculateHash(s string) string {
    h := sha256.New()
    h.Write([]byte(s))
    hashed := h.Sum(nil)
    return hex.EncodeToString(hashed)
}

func CaculateBlockHash(block Block) string {
    record := string(block.Index) + block.Timestamp + string(block.BPM) + block.PrevHash
    return CaculateHash(record)
}

func GenerateBlock(oldBlock Block, BPM int, address string) (Block, error) {
    var newBlock Block
    t := time.Now()

    newBlock.Index = oldBlock.Index + 1
    newBlock.BPM = BPM
    newBlock.Timestamp = t.String()
    newBlock.PrevHash = oldBlock.Hash
    newBlock.Hash = CaculateBlockHash(newBlock)
    newBlock.validator = address

    return newBlock, nil
}

func IsBlockValid(newBlock, oldBlock Block) bool {
    if oldBlock.Index+1 != newBlock.Index {
        return false
    }

    if oldBlock.Hash != newBlock.PrevHash {
        return false
    }

    if CaculateBlockHash(newBlock) != newBlock.Hash {
        return false
    }
    return true
}

The logic of the BlockChain module is relatively simple. We see that the above code implements a total of four methods. CaculateHash, CaculateBlockHash, GenerateBlock, and IsBlockValid calculate string hash, calculate block hash, generate block, and block verification respectively. It should be noted that the validator field in the Block is used to record the node information that generated this block, here is the node name.



##### <b> P2P module implementation </b>
The implementation of the P2P module is realized by the libp2p library of the go language. If you have not understood this part before, it may be relatively difficult to understand. For students who have not been in contact with it before, it is recommended to check the previous article P2P peer-to-peer network first .

In [ ]:
// This is the p2p network, handler the conn and communicate with nodes each other.
// this file is created by magic at 2018-9-2

package dpos

import (
	"bufio"
	"context"
	"crypto/rand"
	"encoding/json"
	"fmt"
	"io"
	mrand "math/rand"
	"os"
	"strconv"
	"strings"
	"sync"
	"time"

	"github.com/davecgh/go-spew/spew"
	"github.com/libp2p/go-libp2p"
	"github.com/libp2p/go-libp2p-core/crypto"
	"github.com/libp2p/go-libp2p-core/network"
	"github.com/libp2p/go-libp2p-core/peer"
	host "github.com/libp2p/go-libp2p-host"
	pstore "github.com/libp2p/go-libp2p-peerstore"
	ma "github.com/multiformats/go-multiaddr"
	"github.com/outbrain/golib/log"
	"github.com/urfave/cli"
)

const (
	// DefaultVote 
	DefaultVote = 10
	// FileName : Node information saving configuration file
	FileName = "config.ini"
)

var mutex = &sync.Mutex{}

//Validator ; Define node information
type Validator struct {
	name string
	vote int
}

// NewNode ; Create a new node to join the P2P network
var NewNode = cli.Command{
	Name:  "new",
	Usage: "add a new node to p2p network",
	Flags: []cli.Flag{
		cli.IntFlag{
			Name:  "port",
			Value: 3000,
			Usage: "The port number of the newly created node",
		},
		cli.StringFlag{
			Name:  "target",
			Value: "",
			Usage: "target node",
		},
		cli.BoolFlag{
			Name:  "secio",
			Usage: "whether to open secio",
		},
		cli.Int64Flag{
			Name:  "seed",
			Value: 0,
			Usage: "Generate random numbers",
		},
	},
	Action: func(context *cli.Context) error {
		if err := Run(context); err != nil {
			return err
		}
		return nil
	},
}

// MakeBasicHost : Build a P2P network
func MakeBasicHost(listenPort int, secio bool, randseed int64) (host.Host, error) {
	var r io.Reader

	if randseed == 0 {
		r = rand.Reader
	} else {
		r = mrand.New(mrand.NewSource(randseed))
	}

	// Generate a pair of public and private keys
	priv, _, err := crypto.GenerateKeyPairWithReader(crypto.RSA, 2048, r)
	if err != nil {
		return nil, err
	}

	opts := []libp2p.Option{
		libp2p.ListenAddrStrings(fmt.Sprintf("/ip4/127.0.0.1/tcp/%d", listenPort)),
		libp2p.Identity(priv),
	}

	if !secio {
		opts = append(opts, libp2p.NoSecurity)
	}
	basicHost, err := libp2p.New(context.Background(), opts...)
	if err != nil {
		return nil, err
	}

	// Build host multiaddress
	hostAddr, _ := ma.NewMultiaddr(fmt.Sprintf("/ipfs/%s", basicHost.ID().Pretty()))

	// Now we can build a full multiaddress to reach this host
	// by encapsulating both addresses;
	addr := basicHost.Addrs()[0]
	fullAddr := addr.Encapsulate(hostAddr)

	log.Infof("I am: %s\n", fullAddr)
	SavePeer(basicHost.ID().Pretty())

	if secio {
		fmt.Printf("Now run the command in a new terminal: './dpos new --port %d --target %s -secio' \n", listenPort+1, fullAddr)
	} else {
		fmt.Printf("Now in a new terminal run the command: './dpos new --port %d --target %s' \n", listenPort+1, fullAddr)
	}
	return basicHost, nil
}

// HandleStream  handler stream info
func HandleStream(s network.Stream) {
	log.Infof("get a new connection: %s", s.Conn().RemotePeer().Pretty())

	rw := bufio.NewReadWriter(bufio.NewReader(s), bufio.NewWriter(s))
	go readData(rw)
	go writeData(rw)
}

// readData Read data output to client
func readData(rw *bufio.ReadWriter) {
	for {
		str, err := rw.ReadString('\n')
		if err != nil {
			log.Errorf(err.Error())
		}

		if str == "" {
			return
		}
		if str != "\n" {
			chain := make([]Block, 0)

			if err := json.Unmarshal([]byte(str), &chain); err != nil {
				log.Errorf(err.Error())
			}

			mutex.Lock()
			if len(chain) > len(BlockChain) {
				BlockChain = chain
				bytes, err := json.MarshalIndent(BlockChain, "", " ")
				if err != nil {
					log.Errorf(err.Error())
				}

				fmt.Printf("\x1b[32m%s\x1b[0m> ", string(bytes))
			}
			mutex.Unlock()
		}
	}
}

// writeData : Write client data processing to BlockChain
func writeData(rw *bufio.ReadWriter) {
	// Start a coroutine to handle terminal synchronization
	// (Tip: Coroutines are generalizations of subroutines)
	go func() {
		for {
			time.Sleep(2 * time.Second)
			mutex.Lock()
			bytes, err := json.Marshal(BlockChain)
			if err != nil {
				log.Errorf(err.Error())
			}
			mutex.Unlock()

			mutex.Lock()
			rw.WriteString(fmt.Sprintf("%s\n", string(bytes)))
			rw.Flush()
			mutex.Unlock()
		}
	}()

	stdReader := bufio.NewReader(os.Stdin)
	for {
		fmt.Print(">")
		sendData, err := stdReader.ReadString('\n')
		if err != nil {
			log.Errorf(err.Error())
		}

		sendData = strings.Replace(sendData, "\n", "", -1)
		bpm, err := strconv.Atoi(sendData)
		if err != nil {
			log.Errorf(err.Error())
		}

		// pick:  selects the block producer
		address := PickWinner()
		log.Infof("******Node %s has obtained bookkeeping rights******", address)
		lastBlock := BlockChain[len(BlockChain)-1]
		newBlock, err := GenerateBlock(lastBlock, bpm, address)
		if err != nil {
			log.Errorf(err.Error())
		}

		if IsBlockValid(newBlock, lastBlock) {
			mutex.Lock()
			BlockChain = append(BlockChain, newBlock)
			mutex.Unlock()
		}

		spew.Dump(BlockChain)

		bytes, err := json.Marshal(BlockChain)
		if err != nil {
			log.Errorf(err.Error())
		}
		mutex.Lock()
		rw.WriteString(fmt.Sprintf("%s\n", string(bytes)))
		rw.Flush()
		mutex.Unlock()
	}
}

// Run 
func Run(ctx *cli.Context) error {

	t := time.Now()
	genesisBlock := Block{}
	genesisBlock = Block{0, t.String(), 0, CaculateBlockHash(genesisBlock), "", ""}
	BlockChain = append(BlockChain, genesisBlock)

	// command line parameter
	port := ctx.Int("port")
	target := ctx.String("target")
	secio := ctx.Bool("secio")
	seed := ctx.Int64("seed")

	if port == 0 {
		log.Fatal("please provide a port number")
	}
	// Construct a host listening address
	ha, err := MakeBasicHost(port, secio, seed)
	if err != nil {
		return err
	}

	if target == "" {
		log.Info("Waiting for node to connect...")
		ha.SetStreamHandler("/p2p/1.0.0", HandleStream)
		select {}
	} else {
		ha.SetStreamHandler("/p2p/1.0.0", HandleStream)
		ipfsaddr, err := ma.NewMultiaddr(target)
		if err != nil {
			return err
		}
		pid, err := ipfsaddr.ValueForProtocol(ma.P_IPFS)
		if err != nil {
			return err
		}

		peerid, err := peer.IDB58Decode(pid)
		if err != nil {
			return err
		}

		targetPeerAddr, _ := ma.NewMultiaddr(
			fmt.Sprintf("/ipfs/%s", peer.IDB58Encode(peerid)))
		targetAddr := ipfsaddr.Decapsulate(targetPeerAddr)

		// Now we have a peerID and a targetaddr, so we add it to the peerstore. Let libP2P know how to connect to it.
		ha.Peerstore().AddAddr(peerid, targetAddr, pstore.PermanentAddrTTL)
		log.Info("打开Stream")

		// Build a new stream from hostB to hostA
		// using the same /p2p/1.0.0 protocol
		s, err := ha.NewStream(context.Background(), peerid, "/p2p/1.0.0")
		if err != nil {
			return err
		}

		rw := bufio.NewReadWriter(bufio.NewReader(s), bufio.NewWriter(s))

		go writeData(rw)
		go readData(rw)
		select {}
	}
	return nil
}

// SavePeer : Save the node information added to the network to the configuration file to facilitate subsequent voting and selection
func SavePeer(name string) {
	vote := DefaultVote // default number of votes
	f, err := os.OpenFile(FileName, os.O_WRONLY|os.O_APPEND|os.O_CREATE, 0666)
	if err != nil {
		log.Errorf(err.Error())
	}
	defer f.Close()

	f.WriteString(name + ":" + strconv.Itoa(vote) + "\n")

}


##### <b>Tools </b>
Node Manage And Vote:

In [ ]:
package tools

import (
	"fmt"
	"os"
	"io/ioutil"
	"strings"
	"strconv"
	"github.com/urfave/cli"
	"github.com/outbrain/golib/log"
	"github.com/csunny/dpos"

)

// NodeVote 节点投票命令
var NodeVote = cli.Command{
	Name: "vote",
	Usage: "vote for node",
	Flags: []cli.Flag{
		cli.StringFlag{
			Name: "name",
			Value: "",
			Usage: "node name", //节点名称
		},
		cli.IntFlag{
			Name: "v",
			Value: 0,
			Usage: "number of votes", //投票数量
		},
	}, 
	Action: func(context *cli.Context) error{
		if err := Vote(context); err != nil{
			return err
		} 
		return nil
	},
}
// Vote for node. The votes of node is origin vote plus new vote.
// votes = originVote + vote 
func Vote(context *cli.Context) error {
	name := context.String("name")
	vote := context.Int("v")

	if name == "" {
		log.Errorf("Node name cannot be empty")
	}

	if vote < 1 {
		log.Errorf("The minimum number of votes is 1")
	}

	f, err := ioutil.ReadFile(dpos.FileName)
	if err != nil {
		log.Errorf(err.Error())
		return err
	}
	res := strings.Split(string(f), "\n")

	voteMap := make(map[string]string)
	for _, node := range res {
		nodeSplit := strings.Split(node, ":")
		if len(nodeSplit) > 1 {
			voteMap[nodeSplit[0]] = fmt.Sprintf("%s", nodeSplit[1])
		}
	}

	originVote, err := strconv.Atoi(voteMap[name])
	if err != nil {
		log.Errorf(err.Error())
		return err
	}
	votes := originVote + vote
	voteMap[name] = fmt.Sprintf("%d", votes)

	log.Infof("节点%s新增票数%d", name, vote)
	str := ""
	for k, v := range voteMap {
		str += k + ":" + v + "\n"
	}

	file, err := os.OpenFile(dpos.FileName, os.O_RDWR, 0666)
	if err != nil{
		return err
	}

	file.WriteString(str)
	defer file.Close()

	return nil
}

##### <b> DPoS module</b>


In [ ]:
package dpos

import (
	"io/ioutil"
	"log"
	"sort"
	"strconv"
	"strings"
)

// BPCount : Number of block producers
const BPCount = 5

// PickWinner : Nodes that generate blocks are selected based on the number of votes
func PickWinner() (bp string) {
	// select BlockProducer

	f, err := ioutil.ReadFile(FileName)
	if err != nil {
		log.Fatal(err)
	}

	res := strings.Split(string(f), "\n")

	voteList := make([]int, len(res))
	voteMap := make(map[string]int)
	for _, node := range res {
		nodeSplit := strings.Split(node, ":")
		if len(nodeSplit) > 1 {
			vote, err := strconv.Atoi(nodeSplit[1])
			if err != nil {
				log.Fatal(err)
			}
			voteList = append(voteList, vote)
			voteMap[nodeSplit[0]] = vote
		}
	}
	sort.Slice(voteList, func(i, j int) bool {
		return voteList[i] > voteList[j]
	})

	if len(voteList) > BPCount {
		voteList = voteList[0:BPCount] // Select the previous 5 nodes as Block producers
	}

	for k, v := range voteMap {
		if v > voteList[len(voteList)-1] {
			bp = k
		}
	}
	return
}


And finally write in the blockchain.

In [ ]:
// implement a simple p2p blockchain which use dpos algorithm.
// this file just for a simple block generate and validate.
// This file is created by magic at 2018-9-2

package dpos

import (
	"crypto/sha256"
	"encoding/hex"
	"time"
)

// BlockChain slice to storage Block
var BlockChain []Block

// Block struct, A block contain :
// Index 索引、Timestamp(时间戳)、BPM、Hash(自己的hash值)、PreHash(上一个块的Hash值)、validator(此区块的生产者信息)
type Block struct {
	Index     int
	Timestamp string
	BPM       int
	Hash      string
	PrevHash  string
	validator string
}

// CaculateHash : Calculate the hash value of string
func CaculateHash(s string) string {
	h := sha256.New()
	h.Write([]byte(s))
	hashed := h.Sum(nil)
	return hex.EncodeToString(hashed)
}

// CaculateBlockHash : Calculate the hash value of the Block
func CaculateBlockHash(block Block) string {
	record := string(block.Index) + block.Timestamp + string(block.BPM) + block.PrevHash
	return CaculateHash(record)
}

// GenerateBlock : Generate a new block based on the previous block information
func GenerateBlock(oldBlock Block, BPM int, address string) (Block, error) {
	var newBlock Block
	t := time.Now()

	newBlock.Index = oldBlock.Index + 1
	newBlock.BPM = BPM
	newBlock.Timestamp = t.String()
	newBlock.PrevHash = oldBlock.Hash
	newBlock.Hash = CaculateBlockHash(newBlock)
	newBlock.validator = address

	return newBlock, nil
}

// IsBlockValid : Check whether the block is legal
func IsBlockValid(newBlock, oldBlock Block) bool {
	if oldBlock.Index+1 != newBlock.Index {
		return false
	}

	if oldBlock.Hash != newBlock.PrevHash {
		return false
	}

	if CaculateBlockHash(newBlock) != newBlock.Hash {
		return false
	}
	return true
}


# Run the app:
To avoid installing anything on your machine, you can test the code using <b> GitPod </b>.
Execute the following commands:

1. git clone git@github.com:csunny/dpos.git
2. cd dpos  
3. go build main/dpos.go

#### connect multi peer
4. ./dpos new --port 3000 --secio

#### Vote
5. ./dpos vote -name QmaxEdbKW4x9mP2vX15zL9fyEsp9b9yV48zwtdrpYddfxe -v 30